<a href="https://colab.research.google.com/github/lustraka/data-analyst-portfolio-project-2022/blob/main/code/20211110_Scrape_Intitial_WebPages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [21]:
csv_path = 'https://raw.githubusercontent.com/lustraka/data-analyst-portfolio-project-2022/main/data/data_analytics_portfolio_projects.csv'
df = pd.read_csv(csv_path)
df.shape

(110, 3)

**2021-11-10**: Clean a csv file and add id (non-repetable):
```python
df = df.iloc[110:]
df['id'] = df.id.apply(lambda x: 'WP'+str(int(x+1)))
df.to_csv('data_analytics_portfolio_projects.csv', index=False)
```

In [22]:
df.head()

,id,title,url
0,WP1,9 Data Analytics Portfolio Examples [2021 Edit...,https://careerfoundry.com/en/blog/data-analyti...
1,WP2,How To Build A Data Analytics Portfolio [Compl...,https://careerfoundry.com/en/blog/data-analyti...
2,WP3,How to Build a Data Analyst Portfolio: Tips fo...,https://www.coursera.org/articles/how-to-build...
3,WP4,How to Build an Impressive Data Analytics Port...,https://www.springboard.com/blog/data-analytic...
4,WP5,Data Analyst Portfolio Project | SQL Data Expl...,https://www.youtube.com/watch%3Fv%3DqfyynHBFOsM


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      110 non-null    object
 1   title   110 non-null    object
 2   url     110 non-null    object
dtypes: object(3)
memory usage: 2.7+ KB


In [47]:
html = []
for row in df.itertuples():
  print(row[1], row[3])
  page = requests.get(row[3])
  if page.status_code == 200:
    html.append([row[1], page.status_code, page.content])
  else:
    html.append([row[1], page.status_code, ''])

html[-1]

WP1 https://careerfoundry.com/en/blog/data-analytics/data-analytics-portfolio-examples/
WP2 https://careerfoundry.com/en/blog/data-analytics/data-analyst-portfolio/
WP3 https://www.coursera.org/articles/how-to-build-a-data-analyst-portfolio
WP4 https://www.springboard.com/blog/data-analytics/data-analyst-portfolio/
WP5 https://www.youtube.com/watch%3Fv%3DqfyynHBFOsM
WP6 https://www.youtube.com/watch%3Fv%3DtW89_4psTWg
WP7 https://www.codecademy.com/resources/blog/data-analyst-portfolio/
WP8 https://www.stratascratch.com/blog/data-analytics-project-ideas-that-will-get-you-the-job/
WP9 https://www.dataquest.io/blog/build-a-data-science-portfolio/
WP10 https://www.kdnuggets.com/2019/04/projects-include-data-science-portfolio.html
WP11 https://www.udacity.com/blog/2016/02/how-to-build-a-data-analysis-portfolio-that-will-get-you-hired.html
WP12 https://towardsdatascience.com/5-of-my-best-data-science-portfolio-projects-8cd7f81f1b1
WP13 https://towardsdatascience.com/these-data-science-portfo

['WP110', 403, '']

In [48]:
html_df = pd.DataFrame(html, columns=['id', 'status', 'content'])
html_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       110 non-null    object
 1   status   110 non-null    int64 
 2   content  110 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.7+ KB


In [50]:
# Check status - probably dynamic pages
html_df.query('status != 200')

,id,status,content
4,WP5,429,
5,WP6,429,
10,WP11,403,
30,WP31,403,
44,WP45,503,
51,WP52,403,
55,WP56,406,
56,WP57,404,
59,WP60,999,
68,WP69,406,


In [53]:
# Merge datasets
df_new = df.merge(html_df, on='id')
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110 entries, 0 to 109
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       110 non-null    object
 1   title    110 non-null    object
 2   url      110 non-null    object
 3   status   110 non-null    int64 
 4   content  110 non-null    object
dtypes: int64(1), object(4)
memory usage: 5.2+ KB


In [58]:
# Add lenght of the content
df_new['cont_len'] = df_new.content.str.len()
df_new.head()

,id,title,url,status,content,cont_len
0,WP1,9 Data Analytics Portfolio Examples [2021 Edit...,https://careerfoundry.com/en/blog/data-analyti...,200,"b'<!doctype html><html lang=en-US prefix=""og: ...",63180
1,WP2,How To Build A Data Analytics Portfolio [Compl...,https://careerfoundry.com/en/blog/data-analyti...,200,"b'<!doctype html><html lang=en-US prefix=""og: ...",45047
2,WP3,How to Build a Data Analyst Portfolio: Tips fo...,https://www.coursera.org/articles/how-to-build...,200,"b'<!DOCTYPE html><html xmlns:fb=""http://ogp.me...",775025
3,WP4,How to Build an Impressive Data Analytics Port...,https://www.springboard.com/blog/data-analytic...,200,"b'<!DOCTYPE html>\n<html lang=""en-US"" class=""n...",97494
4,WP5,Data Analyst Portfolio Project | SQL Data Expl...,https://www.youtube.com/watch%3Fv%3DqfyynHBFOsM,429,,0


In [61]:
# Add date of scaapping - should be part of html requests
df_new['cont_date'] = pd.Timestamp.today()
df_new.head(1)

,id,title,url,status,content,cont_len,cont_date
0,WP1,9 Data Analytics Portfolio Examples [2021 Edit...,https://careerfoundry.com/en/blog/data-analyti...,200,"b'<!doctype html><html lang=en-US prefix=""og: ...",63180,2021-11-10 14:20:34.082045


In [62]:
df_new.set_index('id', inplace=True)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110 entries, WP1 to WP110
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   title      110 non-null    object        
 1   url        110 non-null    object        
 2   status     110 non-null    int64         
 3   content    110 non-null    object        
 4   cont_len   110 non-null    int64         
 5   cont_date  110 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 6.0+ KB


In [63]:
# Store dataframes for further processing
from sqlalchemy import create_engine

# Create SQLAlchemy engine and empty database
engine = create_engine('sqlite:///dapp2022.db')

# Store dataframes in database
df_new.to_sql('wp', engine, index=False)

# Upload the file to GitHub !
# 2021-11-10 :  It has 14 MB

In [65]:
# How many pages have not been loaded?
df_new.query('status != 200').shape[0]

18